In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import StandardScaler, LabelBinarizer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_predict
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from keras import optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop
from tensorflow.keras.metrics import binary_crossentropy, categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight 
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
import time
import scipy
from scipy import ndimage
from sklearn.metrics import confusion_matrix, classification_report
from IPython.core.display import display, HTML
# stop annoying tensorflow warning messages
import logging
import splitfolders
logging.getLogger("tensorflow").setLevel(logging.ERROR)
np.random.seed(123)
print ('modules loaded')


modules loaded


## *Data Pre-Processing*

#### The Data for this project can be downloaded [here](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=52758223)

### The data for this project was acquired in three parts:

> #### 1) A training set separated into 3 directories, each with separate class labeled directories
> #### 2) A validation set with all images in the same directory and a CSV file with filenames and class labels
> #### 3) A test set with no class labels

- #### We will combine all the data of the training and validation sets into labeled directories and split the whole into training, validation, and test sets for use in this project.

- #### We will re-name the provided test set and to "unlabeled" and reserve it for demonstration purposes, as there is no way of knowing if the predicitions are correct and how a partivular model is performing with it.

### Training Data:

In [2]:
# Set paths of 3x2 pre-labeled directories
f0_data_training_ALL_dir = 'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_training_data/fold_0/all/'
f0_data_training_Normal_dir = 'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_training_data/fold_0/hem/'

f1_data_training_ALL_dir = 'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_training_data/fold_1/all/'
f1_data_training_Normal_dir = 'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_training_data/fold_1/hem/'

f2_data_training_ALL_dir = 'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_training_data/fold_2/all/'
f2_data_training_Normal_dir = 'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_training_data/fold_2/hem/'

# Create lists of all ALL images in these directories
f0_imgs_ALL = [file for file in os.listdir(f0_data_training_ALL_dir) if file.endswith('.bmp')]
f1_imgs_ALL = [file for file in os.listdir(f1_data_training_ALL_dir) if file.endswith('.bmp')]
f2_imgs_ALL = [file for file in os.listdir(f2_data_training_ALL_dir) if file.endswith('.bmp')]

# Create lists of all Normal images in these directories
f0_imgs_Normal = [file for file in os.listdir(f0_data_training_Normal_dir) if file.endswith('.bmp')]
f1_imgs_Normal = [file for file in os.listdir(f1_data_training_Normal_dir) if file.endswith('.bmp')]
f2_imgs_Normal = [file for file in os.listdir(f2_data_training_Normal_dir) if file.endswith('.bmp')]


### Validation Data

In [3]:
# Set paths of directory
data_val_dir = 'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_test_prelim_phase_data/'

# Read in and inspect the dataframe with the filenames and labels
val_df = pd.read_csv('data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_test_prelim_phase_data/C-NMC_test_prelim_phase_data_labels.csv')

val_df.head()

,Patient_ID,new_names,labels
0,UID_57_29_1_all.bmp,1.bmp,1
1,UID_57_22_2_all.bmp,2.bmp,1
2,UID_57_31_3_all.bmp,3.bmp,1
3,UID_H49_35_1_hem.bmp,4.bmp,0
4,UID_58_6_13_all.bmp,5.bmp,1


In [4]:
# Change labels to use "binary" class_mode
val_df = val_df.replace({'labels':{0 : 'ALL', 1 : 'Normal'}})

# filenames do not use 'Patient_ID', so drop the column
val_df = val_df.drop('Patient_ID', axis=1)

val_df.head()

,new_names,labels
0,1.bmp,Normal
1,2.bmp,Normal
2,3.bmp,Normal
3,4.bmp,ALL
4,5.bmp,Normal


In [5]:
# Group the the df by labels so we can split them
val_split_df = val_df.groupby('labels')

# Split the filenames into separate df's and create lists of them
val_ALL_df = val_split_df.get_group('ALL')
val_images_ALL = [img for img in val_ALL_df.new_names]

val_Normal_df = val_split_df.get_group('Normal')
val_images_Normal = [img for img in val_Normal_df.new_names]

In [6]:
val_ALL_df.head()

,new_names,labels
3,4.bmp,ALL
6,7.bmp,ALL
7,8.bmp,ALL
12,13.bmp,ALL
13,14.bmp,ALL


In [7]:
val_images_ALL[:5]

['4.bmp', '7.bmp', '8.bmp', '13.bmp', '14.bmp']

In [8]:
val_Normal_df.head()

,new_names,labels
0,1.bmp,Normal
1,2.bmp,Normal
2,3.bmp,Normal
4,5.bmp,Normal
5,6.bmp,Normal


In [9]:
val_images_Normal[:5]

['1.bmp', '2.bmp', '3.bmp', '5.bmp', '6.bmp']

### Combine the data into one directory

In [10]:
# Set new directory path for all images to be combined
new_dir = 'data/labeled_images/'
combo_ALL_dir = os.path.join(new_dir, 'ALL')
combo_Normal_dir = os.path.join(new_dir, 'Normal')

# Create the new directory
os.mkdir(new_dir)
os.mkdir(combo_ALL_dir)
os.mkdir(combo_Normal_dir)

In [11]:
#  Move images from training_data directories:

# ALL
for img in f0_imgs_ALL:
    origin = os.path.join(f0_data_training_ALL_dir, img)
    destination = os.path.join(combo_ALL_dir, img)
    os.rename(origin, destination)
    
for img in f1_imgs_ALL:
    origin = os.path.join(f1_data_training_ALL_dir, img)
    destination = os.path.join(combo_ALL_dir, img)
    os.rename(origin, destination)
    
for img in f2_imgs_ALL:
    origin = os.path.join(f2_data_training_ALL_dir, img)
    destination = os.path.join(combo_ALL_dir, img)
    os.rename(origin, destination)
    
# Normal
for img in f0_imgs_Normal:
    origin = os.path.join(f0_data_training_Normal_dir, img)
    destination = os.path.join(combo_Normal_dir, img)
    os.rename(origin, destination)
    
for img in f1_imgs_Normal:
    origin = os.path.join(f1_data_training_Normal_dir, img)
    destination = os.path.join(combo_Normal_dir, img)
    os.rename(origin, destination)
    
for img in f2_imgs_Normal:
    origin = os.path.join(f2_data_training_Normal_dir, img)
    destination = os.path.join(combo_Normal_dir, img)
    os.rename(origin, destination)

In [12]:
#  Move images from validation_data directory:

# ALL
for img in val_images_ALL:
    origin = os.path.join(data_val_dir, img)
    destination = os.path.join(combo_ALL_dir, img)
    os.rename(origin, destination)

# Normal
for img in val_images_Normal:
    origin = os.path.join(data_val_dir, img)
    destination = os.path.join(combo_Normal_dir, img)
    os.rename(origin, destination)


In [20]:
total = len(os.listdir(combo_ALL_dir)) + len(os.listdir(combo_Normal_dir))
print('There are', len(os.listdir(combo_ALL_dir)) + len(os.listdir(combo_Normal_dir)), 'total images')
print("--------------------------------")
print('There are', len(os.listdir(combo_ALL_dir)), 'ALL cell images')
print('There are', len(os.listdir(combo_Normal_dir)), 'Normal cell images')


There are 12528 total images
--------------------------------
There are 7920 ALL cell images
There are 4608 Normal cell images


### Split the data into training, validation, and test sets
>### Training   ~ 80%
>### Validation ~ 19%
>### Test       ~ 1%

In [41]:
# Create new directory for the training, validation, and testing datasets
tvt_dir = 'data/tvt/'
os.mkdir(tvt_dir)

# Create paths for new dirctory datasets and class label folders
train_folder = os.path.join(tvt_dir, 'train')
train_ALL = os.path.join(train_folder, 'ALL')
train_Normal = os.path.join(train_folder, 'Normal')

val_folder = os.path.join(tvt_dir, 'validation')
val_ALL = os.path.join(val_folder, 'ALL')
val_Normal = os.path.join(val_folder, 'Normal')

test_folder = os.path.join(tvt_dir, 'test')
test_ALL = os.path.join(test_folder, 'ALL')
test_Normal = os.path.join(test_folder, 'Normal')

# Create the class label folders
os.mkdir(train_folder)
os.mkdir(train_ALL)
os.mkdir(train_Normal)

os.mkdir(val_folder)
os.mkdir(val_ALL)
os.mkdir(val_Normal)

os.mkdir(test_folder)
os.mkdir(test_ALL)
os.mkdir(test_Normal)

In [42]:
# do the math for our splits
print('There are', len(os.listdir(combo_ALL_dir)) + len(os.listdir(combo_Normal_dir)), 'total images')
print("--------------------------------")
print('There are', len(os.listdir(combo_ALL_dir)), 'ALL cell images')
print('There are', len(os.listdir(combo_Normal_dir)), 'Normal cell images')

train_ALL_total = (len(os.listdir(combo_ALL_dir))) * 0.80
train_Normal_total = (len(os.listdir(combo_Normal_dir))) * 0.80

val_ALL_total = (len(os.listdir(combo_ALL_dir))) * 0.19
val_Normal_total = (len(os.listdir(combo_Normal_dir))) * 0.19
               
test_ALL_total = (len(os.listdir(combo_ALL_dir))) * 0.01
test_Normal_total = (len(os.listdir(combo_Normal_dir))) * 0.01
print("--------------------------------")
print("--------------------------------")
print("Train ALL        :", round(train_ALL_total,2), "  :", )
print("Train Normal     :", round(train_Normal_total,2), "  :", )
print("--------------------------------")
print("Validation ALL   :", round(val_ALL_total,2),   "  :", val_ALL_total+train_ALL_total)
print("Validation Nomral:", round(val_Normal_total,2),   "  :", val_Normal_total+train_Normal_total)
print("--------------------------------")
print("Test ALL         :", round(test_ALL_total,2),  "    :", val_ALL_total+train_ALL_total+test_ALL_total)
print("Test Normal      :", round(test_Normal_total,2),  "   :", val_Normal_total+train_Normal_total+test_Normal_total)

There are 12528 total images
--------------------------------
There are 7920 ALL cell images
There are 4608 Normal cell images
--------------------------------
--------------------------------
Train ALL        : 6336.0   :
Train Normal     : 3686.4   :
--------------------------------
Validation ALL   : 1504.8   : 7840.8
Validation Nomral: 875.52   : 4561.92
--------------------------------
Test ALL         : 79.2     : 7920.0
Test Normal      : 46.08    : 4608.0


### Create the Splits

In [43]:
# Create iterables
imgs_ALL = [img for img in os.listdir(combo_ALL_dir)]
imgs_Normal = [img for img in os.listdir(combo_Normal_dir)]

# train ALL
imgs = imgs_ALL[:6337]
for img in imgs:
    origin = os.path.join(combo_ALL_dir, img)
    destination = os.path.join(train_ALL, img)
    shutil.copyfile(origin, destination)
    
# validation ALL
imgs = imgs_ALL[6337:7841]
for img in imgs:
    origin = os.path.join(combo_ALL_dir, img)
    destination = os.path.join(val_ALL, img)
    shutil.copyfile(origin, destination)

# test ALL
imgs = imgs_ALL[7841:]
for img in imgs:
    origin = os.path.join(combo_ALL_dir, img)
    destination = os.path.join(test_ALL, img)
    shutil.copyfile(origin, destination)
    
# train Normal
imgs = imgs_Normal[:3687]
for img in imgs:
    origin = os.path.join(combo_Normal_dir, img)
    destination = os.path.join(train_Normal, img)
    shutil.copyfile(origin, destination)

# validation Normal
imgs = imgs_Normal[3687:4562]
for img in imgs:
    origin = os.path.join(combo_Normal_dir, img)
    destination = os.path.join(val_Normal, img)
    shutil.copyfile(origin, destination)
    
# test Normal
imgs = imgs_Normal[4562:]
for img in imgs:
    origin = os.path.join(combo_Normal_dir, img)
    destination = os.path.join(test_Normal, img)
    shutil.copyfile(origin, destination)

In [44]:
# Reserve unlabeled data
origin_path =  'data/PKG - C-NMC_Leukemia/C-NMC_Leukemia/C-NMC_test_final_phase_data/'
destination_path = 'data/unlabeled_images/'
os.mkdir(destination_path)

unlabeled_images = [img for img in os.listdir(origin_path)]

# Move unlabeled images to the new directory
for img in unlabeled_images:
    origin = os.path.join(origin_path, img)
    destination = os.path.join(destination_path, img)
    os.rename(origin, destination)

In [46]:
#Remove unnecessary directories from the 'data' file
shutil.rmtree('data/PKG - C-NMC_Leukemia/')